In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

from pprint import pprint
import json

In [3]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

c:\Users\IloveKOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-QISotLL5-py3.11\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model,
)

In [5]:
chroma_db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [6]:
from langchain_core.documents import Document

documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.",
]

doc_objects = []

for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_Textbook", "chapter": f"Chapter {i}"}
    )

    doc_objects.append(doc)

#순차적 ID 리스트 생성
doc_ids = [f"doc_{i}" for i in range(1, len(doc_objects) + 1)]

added_docs_ids = chroma_db.add_documents(documents = doc_objects, ids=doc_ids)

print(f"{len(added_docs_ids)} 개의 문서가 성공적으로 DB에 추가되었습니다.")
print(added_docs_ids)

5 개의 문서가 성공적으로 DB에 추가되었습니다.
['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5']


In [7]:
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2) #관련 문서 k개 추출, 없으면 0도 가능

print(f"질문: {query}")
for doc in results:
    print(f"문서 ID: {doc.metadata['chapter']}, 내용: {doc.page_content}")

질문: 인공지능과 머신러닝의 관계는?
문서 ID: Chapter 2, 내용: 머신러닝은 인공지능의 하위 분야입니다.
문서 ID: Chapter 3, 내용: 딥러닝은 머신러닝의 한 종류입니다.


In [8]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
# chroma_db.update_documents(document_ids=added_docs_ids, documents=doc_objects, document=doc_objects)
# Document(page_content, metadata={})

update_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다",
    metadata={"source": "AI_Textbook", "chapter": "Chapter 6"}
)

chroma_db.update_document(
    document_id="doc_1",
    document=update_document_1
)

print("문서 업데이트 완료")

문서 업데이트 완료


In [10]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4', 'doc_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [11]:
chroma_db.delete(ids=["doc_5"])

In [12]:
chroma_db.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [13]:
chroma_db2 = Chroma(
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
    embedding_function=embeddings_model
)

chroma_db2.get()

{'ids': ['doc_1', 'doc_2', 'doc_3', 'doc_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야로, 머신러닝과 딥러닝을 포함합니다',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 6', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_Textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_Textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

In [15]:
# 데이터 로드
def load_text_files(text_files):
    data = []
    for file in text_files:
        loader = TextLoader(file, encoding="utf-8")
        data += loader.load()
    
    return data

korean_text_files = glob(os.path.join("data", "*_KR.txt"))
korean_data = load_text_files(korean_text_files)

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    separator = r"[.!?]\s+",
    chunk_size = 100,
    chunk_overlap = 0,
    is_separator_regex = True,
    keep_separator = True,
)


korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수: ", len(korean_docs))

한국어 문서 수:  8


In [16]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings_huggingface = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

chroma_db = Chroma.from_documents(
    documents=korean_docs,
    embedding=embeddings_huggingface,
    collection_name = "db_korean_cosine",
    persist_directory="./chroma_db",
    collection_metadata={"hnsw:space": "cosine"}, #12, ip, cosine
)

chroma_db.get()

{'ids': ['ab931cdc-cbb7-44f4-87c9-072240e7c5de',
  '4d1ec4d3-22fe-4eff-bacf-7dfef9d21a0b',
  'ece0b64f-55bc-4079-8857-2caa71d46805',
  '06558093-514c-4ad3-a63f-af6505a583aa',
  '7e4ab40c-807a-4294-a9d1-5748b5bf3d34',
  '646acb5d-71f6-4083-8faf-0bd8b192d3f3',
  '44497e91-9d2d-48a7-9156-b90920184032',
  '2ca8e234-6956-4ee4-b5ff-3d15acbc2afc'],
 'embeddings': None,
 'documents': ['리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다',
  '.\n\n리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다',
  '. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다',
  '.\n\n리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.',
  '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크

In [17]:
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색결과")
for doc in retrieved_docs:
    print(f" - {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\리비안_KR.txt]
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data\리비안_KR.txt]


### Similarity Score Threshold(기준 스코어 이상인 문서를 대상으로 추출)

In [42]:
from langchain_community.utils.math import cosine_similarity

chroma_mmr = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 2,
        "score_threshold": 0.6, # 유사도 임계값
    },
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색결과")
for doc in retrieved_docs:
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)],
        [embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f" - {doc.page_content} (유사도: {score:.4f})")
    
    print("-" * 50)

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 (유사도: 0.6734)
--------------------------------------------------


In [51]:
chroma_metadata = chroma_db.as_retriever(
    search_kwargs = {
        "k": 2,
        "filter": {"source": "data\\리비안_KR.txt"}, #메타데이터 필터링
    }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_metadata.invoke(query)

print(f"쿼리: {query}")
print("검색결과")
for doc in retrieved_docs:
    print(f" - {doc.page_content} (유사도: {doc.metadata['source']})")
    print("-" * 50)

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 (유사도: data\리비안_KR.txt)
--------------------------------------------------
 - .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 (유사도: data\리비안_KR.txt)
--------------------------------------------------


In [53]:
chroma_content = chroma_db.as_retriever(
    search_kwargs = {
        "k": 2,
        "where_document": {"$contains": "테슬라"}, #메타데이터 필터링
    }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print("검색결과")
for doc in retrieved_docs:
    print(f" - {doc.page_content} (유사도: {doc.metadata['source']})")
    print("-" * 50)

쿼리: 리비안은 언제 사업을 시작했나요?
검색결과
 - 테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다 (유사도: data\테슬라_KR.txt)
--------------------------------------------------
 - . 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다 (유사도: data\테슬라_KR.txt)
--------------------------------------------------


#### 챗봇

In [88]:
retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

In [89]:
# RAG 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [90]:
retriever_chain = retriever | format_docs

In [101]:
# LLM 체인 생성
rag_chain = (
    {"context": retriever_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [103]:
import gradio as gr
from langchain_core.messages import HumanMessage, AIMessage

def answer_invoke(message, history):
    history_langchain = []
    for human, ai in history:
        history_langchain.append(HumanMessage(content=human))
        history_langchain.append(AIMessage(content=ai))

    history_langchain.append(HumanMessage(content=message))

    response = rag_chain.invoke(message)

    final_answer = llm.invoke(
        history_langchain[:-1] + [AIMessage(content=response)] + [HumanMessage(content=message)]
    )
    return final_answer.content

demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")
demo.launch()


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


c:\Users\IloveKOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-QISotLL5-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [104]:
demo.close()

Closing server running on port: 7864
